In [1]:
# Libraries

import pandas as pd
import numpy as np
import re
from pprint import pprint
from sklearn.manifold import TSNE
from scipy.cluster.hierarchy import fcluster
from sklearn.preprocessing import normalize
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reading CSV files

optim_f_df=pd.read_csv('data/BGEIDSC.EF2EFFP.csv')                   # fuel optimization 
trans_f_df=pd.read_csv('data/BGETCHDATA.PTCHTRANH.csv')              # fuel transactions
event_f_df=pd.read_csv('data/Extranet2.ExactFuelEvents.csv')         # fuel events
level_f_df=pd.read_csv('data/Extranet2.ExactFuelTankLevels.csv')     # fuel levels
truck_v_df=pd.read_csv('data/IBGEFILE.UNITS.csv')                    # trucks 
perfo_v_df=pd.read_csv('data/Extranet2.QCPerformanceExtracts.csv')   # vehicle performance


### Overviwing Data

In [3]:
optim_f_df.head(3)

# TRUCK     Unit Number               *****                                           [TU]
# ACTIVE    Active Flag (for fuel optimization)
# SENDFUEL  Send Fuel Flag (for fuel optimization)
# SENDROUTE Send Route Flag (for fuel optimization)
# TANKCAP   Tank Capacity (gallons)                                       * 5 types
# AVGMPG    MPG used for Fuel optimization (real value is closer to 7mpg)


,TRUCK,ACTIVE,SENDFUEL,SENDROUTE,TANKCAP,AVGMPG,SATTYPE
0,1,Y,Y,Y,200,6.0,
1,243,Y,Y,Y,200,6.0,
2,244,Y,Y,Y,200,6.0,


In [4]:
# This block creates a list of truck IDs as numeric values (valid_trucks).

good_trux_3 = optim_f_df['TRUCK']

list_of_trucks_3 = list(set(good_trux_3))

valid_trucks_o = [x for x in list_of_trucks_3 if (x >= 1 | x < 9999)]

In [5]:
trans_f_df.head(3)

# TRNID   Transaction ID 
# TRNDAT  Transaction Date (YYYYMMDD)
# TRNTIM  Transaction Time (HHMM)
# TRNTS#  Station ID
# TRNTSN  Station Name
# TRNSTC  Station City
# TRNST   Station State
# TRNDRI  Driver Code                                                          [DR]
# TRNUNT  Unit ID                                                              [TU]
# TRNPI1  Item Code 1
# TRNPQ1  Item Quantity 1
# TRNPI2  Item Code 2
# TRNPQ2  Item Quantity 2
# TRNPI3  Item Code 3
# TRNPQ3  Item Quantity 3
# TRNPI4  Item Code 4
# TRNPQ4  Item Quantity 4
# TRNPI5  Item Code 5
# TRNPQ5  Item Quantity 5
# TRNPI6  Item Code 6
# TRNPQ6  Item Quantity 6

# Actual fuel transactions will have an item code of ULSD or FUEL, CDSL, DSL1, BDSL, 
# and could be in any of the Item slots 1-6


# *********** TASK: Filter the valid fuel items and qty, then add them up in an additional column.


,TRNID,TRNDAT,TRNTIM,TRNTS#,TRNTSN,TRNSTC,TRNST,TRNDRI,TRNUNT,TRNPI1,...,TRNPI2,TRNPQ2,TRNPI3,TRNPQ3,TRNPI4,TRNPQ4,TRNPI5,TRNPQ5,TRNPI6,TRNPQ6
0,42574765,20170101,20,520277,FLYING J VANDALIA 97,VANDALIA,OH,ULAL,1564,SCLE,...,,0.00,,0.0,,0.0,,0.0,,0
1,42575331,20170101,48,516202,PILOT NEW BRAUNFELS 330,NEW BRAUNFELS,TX,MANRI,1584,ULSD,...,DEFD,7.71,CADV,1.0,,0.0,,0.0,,0
2,42578678,20170101,134,516277,PILOT 421,DALTON,GA,RUSR,1713,ULSD,...,DEFD,4.12,,0.0,,0.0,,0.0,,0


In [6]:
# This block creates a list of truck IDs as numeric values (valid_trucks).

good_trux = trans_f_df['TRNUNT'].str.extract('(\d+)')

list_of_trucks = list(set(good_trux))

num_trux = pd.to_numeric(list_of_trucks, downcast='integer')

valid_trucks_t = [x for x in num_trux if np.isnan(x) != True]



/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# This block creates a list of driver codes.

good_driv = trans_f_df['TRNDRI'].str.extract('(\S+)')

list_of_drivs = list(set(good_driv))

valid_driv_t = [x for x in list_of_drivs if str(x) != 'nan']


/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
# We don't need the TRNPI 2-6 column since they do not contain any valid fuel item (I have checked them 
# but the queries are not here.)

trans_f_df['TRNPI1'].unique()

array(['SCLE', 'ULSD', 'CADV', 'DEFD', 'ADD ', 'OIL ', 'UREA', 'FUEL',
       'CDSL', 'DEFC', 'DSL1', 'BDSL'], dtype=object)

In [9]:
level_f_df.head(3)

# Id                AutoNumber id
# TankId            Tank Sensor #         (for our trucks, will always be 1)
# TankLevelPercent  Percentage Reading    0-100.00
# TankLevelGallons  Gallons               (Percentage * Tank Capacity) (calculated)
# ExactFuelEventId  Foreign Key to ExactFuelEvent                                       [EV]

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.6,12667508
1,10391482,1,72.4,144.8,12667510
2,10391505,1,57.1,114.2,12667533


In [10]:
level_f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7908828 entries, 0 to 7908827
Data columns (total 5 columns):
Id                  int64
TankId              int64
TankLevelPercent    float64
TankLevelGallons    float64
ExactFuelEventId    int64
dtypes: float64(2), int64(3)
memory usage: 301.7 MB


In [11]:
event_f_df.head(3)

# Id                AutoNumber ID                                                   [EV]
# ESS_Id            Omnitracs Event Subscriber Service Event Id
# EventTimeStamp    DateTime of Event (normalized to Central Time)
# EquipmentID       Unit Number                                                     [TU] (?)
# MCTNumber         Mobile Communications Terminal ID
# EquipmentType     should always read ‘tractor’
# DriverId          Driver Code Assigned to unit at time of reading (varchar(6))    [DR] (?)
# Latitude          decimal degrees latitude at time of reading
# Longitude         decimal degrees longitude at time of reading
# LocationTimeStamp DateTime of location reading
# Speed             MPH at time of reading
# Heading           direction of travel at time of reading
# Odometer          unit odometer at time of reading
# IgnitionStatus    1=on, 2=off
# EFReportReason    0=ignition on, 1=ignition off, 2=timer

# NOTE: Discard equipment IDs that are wrongfully replaced by MCTNumber


,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
0,12649083,13640241,2017-01-01 00:00:32.387,1992,105369677,tractor,BATK,30.34642,-81.70724,2017-01-01 00:00:30.000,0,0.0,39895.5,2,1
1,12649084,13640242,2017-01-01 00:01:24.867,1712,105437938,tractor,TAYANT,36.93953,-84.09541,2017-01-01 00:01:27.000,0,0.0,360378.8,1,2
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2


In [12]:
# This block creates a list of truck IDs as numeric values (valid_trucks).

good_trux_2 = event_f_df['EquipmentID']

list_of_trucks_2 = list(set(good_trux_2))

valid_trucks_e = [x for x in list_of_trucks_2 if (x >= 1 | x < 9999)]

In [13]:
# This block creates a list of driver codes.

good_driv_2 = event_f_df['DriverID'].str.extract('(\S+)')

list_of_drivs_2 = list(set(good_driv_2))

valid_driv_e = [x for x in list_of_drivs_2 if str(x) != 'nan']

/Users/mac/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
truck_v_df.head(3)

# UNUNIT   Unit Number                                      [TU]
# UNYEAR   Model Year
# UNMAKE   Make

,UNUNIT,UNYEAR,UNMAKE
0,535,1996,PETERBILT
1,534,1996,PETERBILT
2,533,1996,PETERBILT


In [15]:
truck_v_df.UNUNIT.value_counts(dropna = False)

1       2
1937    1
1622    1
1969    1
2079    1
2111    1
2133    1
1904    1
2169    1
304     1
1773    1
1590    1
2047    1
1664    1
1915    1
2013    1
1558    1
1685    1
1881    1
1617    1
1851    1
1576    1
1621    1
2082    1
1532    1
2109    1
246     1
1533    1
1800    1
2117    1
       ..
2031    1
1869    1
2028    1
1689    1
1784    1
1879    1
533     1
1641    1
1829    1
2116    1
1843    1
1929    1
2156    1
1992    1
1511    1
2098    1
2119    1
2128    1
1963    1
2008    1
2006    1
1765    1
2142    1
1898    1
1567    1
1844    1
1834    1
1978    1
1513    1
1747    1
Name: UNUNIT, Length: 708, dtype: int64

In [16]:
perfo_v_df.head(3)

# See Mary's post on slack 
# https://nashvillesoftware.slack.com/files/U71DCPYBX/FAF8Q7JSZ/screen_shot_2018-04-28_at_9.03.29_am.png

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Driver2ID,Latitude,Longitude,...,ExcessSpeedTime,TotalFuelUsed,IdleFuelUsed,FaultFlag,RegisteredDriver,CruiseControlTime,TopGearTime,GearDataSource,SpeedUnits,SpeedMatrix
0,76818,13641356,2017-01-01 07:29:49.000,1995,105356577,tractor,LYNS,NaN,36.066249,-86.434814,...,0,0.3,0.3,0,0,0,0,3,MPH,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
1,76819,13641357,2017-01-01 07:29:50.000,1995,105356577,tractor,LYNS,NaN,36.066249,-86.434814,...,0,239.7,21.2,0,1,49,1399,3,MPH,"301,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,..."
2,76820,13641432,2017-01-01 07:49:44.000,105420005,105420005,tractor,LOTG,NaN,36.195138,-83.174583,...,0,246.1,3.2,0,1,1014,1362,3,MPH,"62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0..."


##### Downsizing the databases
To do this, we eliminate all drivers/trucks that have invalid syntax, followed by reducing the databases that contain these elements

In [141]:
# Here all the valid drivers and trucks are inner-joined (intersected), to be used for downsizing the database.

valid_driv = [x for x in valid_driv_t if (x in valid_driv_e)]

# 2 step inner join for tricks
valid_truck = [x for x in valid_trucks_e if (x in valid_trucks_o)]
valid_trux = [x for x in valid_truck if (x in valid_trucks_t)]

In [142]:
# Downsizing dataframes take place here.

# optimization
optim_f_df_ds = optim_f_df[optim_f_df.TRUCK.isin(valid_trux) == True]
# events
event_f_df_ds = event_f_df[(event_f_df.EquipmentID.isin(valid_trux) == True) & (event_f_df.DriverID.isin(valid_driv) == True)]
# Transactions
trans_f_df['numtruc'] = pd.to_numeric(trans_f_df['TRNUNT'], errors='coerce')
trans_f_df_ds = trans_f_df[(trans_f_df.TRNDRI.isin(valid_driv) == True) & (trans_f_df.numtruc.isin(valid_trux) == True)]

### Merging data sets and cleaning columns

In [143]:
fuel_trak_df = pd.merge(event_f_df_ds, level_f_df, how='inner', left_on='Id', right_on='ExactFuelEventId')

In [146]:
event_f_df_ds.head(2)

,Id,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason
2,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2
3,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2


In [147]:
level_f_df.head(3)

,Id,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,10391480,1,82.8,165.6,12667508
1,10391482,1,72.4,144.8,12667510
2,10391505,1,57.1,114.2,12667533


In [145]:
fuel_trak_df.head()

,Id_x,ESS_Id,EventTimeStamp,EquipmentID,MCTNumber,EquipmentType,DriverID,Latitude,Longitude,LocationTimeStamp,Speed,Heading,Odometer,IgnitionStatus,EFReportReason,Id_y,TankId,TankLevelPercent,TankLevelGallons,ExactFuelEventId
0,12649092,13640251,2017-01-01 00:04:15.053,1713,105438293,tractor,RUSR,35.58664,-84.52786,2017-01-01 00:04:17.000,67,219.5,377577.4,1,2,10375458,1,31.7,63.4,12649092
1,12649102,13640261,2017-01-01 00:04:10.260,1585,105301976,tractor,MCAD,35.15430,-90.14263,2017-01-01 00:04:10.000,0,0.0,513931.8,1,2,10375468,1,43.2,108.0,12649102
2,12649104,13640273,2017-01-01 00:11:08.123,1711,105321298,tractor,FOSJA,30.51558,-81.63283,2017-01-01 00:11:10.000,3,127.6,342948.0,1,2,10375470,1,81.7,163.4,12649104
3,12649105,13640274,2017-01-01 00:11:48.803,1693,105356123,tractor,THOCHA,36.18582,-86.26805,2017-01-01 00:11:50.000,0,333.2,340043.7,1,2,10375471,1,37.4,74.8,12649105
4,12649106,13640275,2017-01-01 00:12:26.077,1787,105455670,tractor,RODMAR,35.16910,-80.88830,2017-01-01 00:12:27.000,53,193.1,224306.6,1,2,10375472,1,62.5,125.0,12649106


In [148]:
event_f_df_ds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4057083 entries, 2 to 8495129
Data columns (total 15 columns):
Id                   int64
ESS_Id               int64
EventTimeStamp       object
EquipmentID          int64
MCTNumber            int64
EquipmentType        object
DriverID             object
Latitude             float64
Longitude            float64
LocationTimeStamp    object
Speed                int64
Heading              float64
Odometer             float64
IgnitionStatus       int64
EFReportReason       int64
dtypes: float64(4), int64(7), object(4)
memory usage: 495.2+ MB


In [149]:
level_f_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7908828 entries, 0 to 7908827
Data columns (total 5 columns):
Id                  int64
TankId              int64
TankLevelPercent    float64
TankLevelGallons    float64
ExactFuelEventId    int64
dtypes: float64(2), int64(3)
memory usage: 301.7 MB


In [150]:
fuel_trak_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3858564 entries, 0 to 3858563
Data columns (total 20 columns):
Id_x                 int64
ESS_Id               int64
EventTimeStamp       object
EquipmentID          int64
MCTNumber            int64
EquipmentType        object
DriverID             object
Latitude             float64
Longitude            float64
LocationTimeStamp    object
Speed                int64
Heading              float64
Odometer             float64
IgnitionStatus       int64
EFReportReason       int64
Id_y                 int64
TankId               int64
TankLevelPercent     float64
TankLevelGallons     float64
ExactFuelEventId     int64
dtypes: float64(6), int64(10), object(4)
memory usage: 618.2+ MB


In [85]:
type(truck_v_df.UNUNIT[9])

str

# Data Dictionary
 Fuel Analysis Project
 
###### BGETCHDATA.PTCHTRANH

1.	TRNID		Transaction ID 
2.	TRNDAT	    Transaction Date (YYYYMMDD)
3.	TRNTIM		Transaction Time (HHMM)
4.	TRNTS#		Station ID
5.	TRNTSN		Station Name
6.	TRNSTC		Station City
7.	TRNST		Station State
8.	TRNDRI		Driver Code
9.	TRNUNT	    Unit ID
10.	TRNPI1		Item Code 1
11.	TRNPQ1	    Item Quantity 1
12.	TRNPI2		Item Code 2
13.	TRNPQ2	    Item Quantity 2
14.	TRNPI3		Item Code 3
15.	TRNPQ3	    Item Quantity 3
16.	TRNPI4		Item Code 4
17.	TRNPQ4	    Item Quantity 4
18.	TRNPI5		Item Code 5
19.	TRNPQ5	    Item Quantity 5
20.	TRNPI6		Item Code 6
21.	TRNPQ6      Item Quantity 6

Actual fuel transactions will have an item code of ULSD or FUEL, CDSL, DSL1, BDSL, and could be in any of the Item slots 1-6
 

###### IBGEFILE.UNITS

1.	UNUNIT	Unit Number
2.	UNYEAR	Model Year
3.	UNMAKE	Make
4.	UNSER	VIN

###### BGEIDSC.EF2EFFP

1.	TRUCK		Unit Number
2.	ACTIVE		Active Flag (for fuel optimization)
3.	SENDFUEL	Send Fuel Flag (for fuel optimization)
4.	SENDROUTE	Send Route Flag (for fuel optimization)
5.	TANKCAP	    Tank Capacity (gallons)
6.	AVGMPG	    MPG used for Fuel optimization (real value is closer to 7mpg)

###### Extranet2.ExactFuelEvent

1.	Id			    AutoNumber ID
2.	ESS_Id			Omnitracs Event Subscriber Service Event Id
3.	EventTimeStamp	DateTime of Event (normalized to Central Time)
4.	EquipmentID		Unit Number
5.	MCTNumber		Mobile Communications Terminal ID
6.	EquipmentType	should always read ‘tractor’
7.	DriverId		Driver Code Assigned to unit at time of reading (varchar(6))
8.	Latitude		decimal degrees latitude at time of reading
9.	Longitude		decimal degrees longitude at time of reading
10.	LocationTimeStamp 	DateTime of location reading
11.	Speed			MPH at time of reading
12.	Heading		    direction of travel at time of reading
13.	Odometer		unit odometer at time of reading
14.	IgnitionStatus	1=on, 2=off
15.	EFReportReason	0=ignition on, 1=ignition off, 2=timer

###### Extranet2.ExactFuelTankLevel

1.	Id			        AutoNumber id
2.	TankId			    Tank Sensor # (for our trucks, will always be 1)
3.	TankLevelPercent	Percentage Reading 0-100.00
4.	TankLevelGallons	Gallons (Percentage * Tank Capacity) (calculated)
5.	ExactFuelEventId	Foreign Key to ExactFuelEvent

##### Extranet2.QCPerformanceExtracts
See Mary's post on slack 
https://nashvillesoftware.slack.com/files/U71DCPYBX/FAF8Q7JSZ/screen_shot_2018-04-28_at_9.03.29_am.png

# STEPS TO TAKE
+ Identify invalid Trucks/Driver names
+ Convert into valid syntax (numeric etc)

TO DO:
+ Make an intersection of all Trucks & Drivers and create a final list of good IDs for trucks and drivers

- Extract records with only "good" IDs (downsizing the database)

- Merge dataframes (subject to discussion)

- Group by Driver X Truck
- Sort By time
- Calculate the shift in ODO and fuel level